# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Открываем файл с данными и изучаем общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('D://Игорь/projects/data.csv')
df.info()
#рассмотрим основную информацию о датасете.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### **Вывод:**
- У нас в датасете 12 столбцов.
- Количеством детей, если смотреть только первые 10 значений - все актуально.
- Количество отработанных дней. Здесь сразу видны отриательные значения.
- Возраст клиента.
- Образование. Обратим внимание на то что образование написано в разных регистрах.
- Идентификатор образования.
- Семейное положение. (проверить на регистр)
- Идентификатор семейного положения.
- Пол.
- Сфера деятельности.
- Наличие задолженности.
- Доход.
- Цель кредита.

## Предобработка данных

### Обработка пропусков

In [3]:
print(df.isna().mean())
# у нас имеются пропущенные значиения в столбце с количеством отработанных дней и доходом. 
print(df['days_employed'].head(10))
#есть предположение что положительные значения в часах а не в днях, т.к. 340 266 / 24 даст ответ 14 177, что более логично.
def fixed_days_employed(row):
    if row> 0:
        row = row / 24
        return row
    elif row < 0:
        row = row * -1
        return row
    else:
        return row
df.dropna(inplace=True)

df['days_employed'] = df['days_employed'].apply(fixed_days_employed)
print (df['days_employed'].head(10))
#df['days_employed'] = df['days_employed'].fillna(0)
#и повторно выведем на экран
print(df.isna().mean())

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64
0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
5      -926.185831
6     -2879.202052
7      -152.779569
8     -6929.865299
9     -2188.756445
Name: days_employed, dtype: float64
0     8437.673028
1     4024.803754
2     5623.422610
3     4124.747207
4    14177.753002
5      926.185831
6     2879.202052
7      152.779569
8     6929.865299
9     2188.756445
Name: days_employed, dtype: float64
children            0.0
days_employed       0.0
dob_years           0.0
education           0.0
education_id        0.0
family_status       0.0
family_status_id    0.0
gender              0.0
inc

In [4]:
# теперь необходимо убрать пропуски в доходе.Проще всего будет заменить их на 0, но т.к. их больше 10% лучше будет поменять
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))


In [5]:
#проверим необходимые столбцы на уникальные значения
print('уникальные days_employed :', df['days_employed'].unique())
print('уникальные education :', df['education'].unique())
print('уникальные family_status :', df['family_status'].unique())
print('уникальные gender :', df['gender'].unique())
print('уникальные income_type :', df['income_type'].unique())
print('уникальные total_income :', df['total_income'].unique())
print('уникальные purpose :', df['purpose'].unique())
print('уникальные children :', df['children'].unique())


уникальные days_employed : [8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]
уникальные education : ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
уникальные family_status : ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
уникальные gender : ['F' 'M' 'XNA']
уникальные income_type : ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный' 'студент'
 'предприниматель' 'в декрете']
уникальные total_income : [253875.6394526  112080.01410244 145885.95229686 ...  89672.56115303
 244093.05050043  82047.41889948]
уникальные purpose : ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупк

In [6]:
print(df[df['gender'] == 'XNA'])
# у нас имеется одна строчка с неизвестным гендером... 
df = df.loc[df['gender'] != 'XNA'] #Убираем строку с неизвестным гендером
print(df.head(10))


       children  days_employed  dob_years            education  education_id  \
10701         0    2358.600502         24  неоконченное высшее             2   

          family_status  family_status_id gender income_type  debt  \
10701  гражданский брак                 1    XNA   компаньон     0   

        total_income               purpose  
10701  203905.157261  покупка недвижимости  
   children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   Среднее             1   
3         3    4124.747207         32   среднее             1   
4         0   14177.753002         53   среднее             1   
5         0     926.185831         27    высшее             0   
6         0    2879.202052         43    высшее             0   
7         0     152.779569         50   СРЕДНЕЕ             1   
8         2    6929.865

In [7]:
# в столбце education и family status переведем все значения в строчные
df['education']=df['education'].str.lower()
df['family_status']=df['family_status'].str.lower()


#проверим уникальные значения этих двух столбцов повторно
print('уникальные education :', df['education'].unique())
print('уникальные family_status :', df['family_status'].unique())
#либо так




уникальные education : ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
уникальные family_status : ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


In [8]:
df['education'].value_counts() #либо так

среднее                13693
высшее                  4716
неоконченное высшее      674
начальное                261
ученая степень             6
Name: education, dtype: int64

In [9]:
df['family_status'].value_counts() #и так

женат / замужем          11143
гражданский брак          3734
не женат / не замужем     2525
в разводе                 1083
вдовец / вдова             865
Name: family_status, dtype: int64

In [10]:
#В столбце с количеством детей есть значение 20. вероятнее всего это ошибка. Заменим его медианой 
#и переведем сразу все значения из отрицательныхв положительные
df['children'] = df['children'].replace(20, df['children'].median())
df['children'] = abs(df['children']).astype('int')
print('уникальные children :', df['children'].unique())

уникальные children : [1 0 3 2 4 5]


#### **Вывод:**

- Были пропуски в доходе и количестве отработанных дней. 
- Количесчтво отработанных дней мы заменили на 0 т.к. на анализ это значение не влияет. 
- Пропущеные значения с доходом мы заменили на медиану, т.к. для анализа данное значение играет роль. 
- В данных об обрпазовании и семейном положении были значения в разнгых регистрах. 
- Так же была одна строчка с неизвестным полом. Ее удалили. 
- В столбце с количеством детей было значение 20. 
- Вероятнее всего это опечатка при заполнении данных. ЗАменили на медиану. 
- И отрицательное значение детей перевели в положительное.

### Замена типа данных

In [11]:
#заменим количество отработанных дней на целые числа т.к. отработанные дни могут быть только целыми и 
# зп тоже в целые для удобства работы с ними
df['days_employed']=df['days_employed'].astype('int')
df['total_income']=df['total_income'].astype('int')
#и проверим
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19350 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          19350 non-null  int64 
 1   days_employed     19350 non-null  int64 
 2   dob_years         19350 non-null  int64 
 3   education         19350 non-null  object
 4   education_id      19350 non-null  int64 
 5   family_status     19350 non-null  object
 6   family_status_id  19350 non-null  int64 
 7   gender            19350 non-null  object
 8   income_type       19350 non-null  object
 9   debt              19350 non-null  int64 
 10  total_income      19350 non-null  int64 
 11  purpose           19350 non-null  object
dtypes: int64(7), object(5)
memory usage: 1.9+ MB


#### **Вывод:**
Заменили на целые числа доход и количество отработанных дней, это удобно для обработки данных и будет корректнее. (по крайней мере к количеству ортработанных дненй, т.к. их может быть только целое количество)

### Обработка дубликатов

In [12]:
df.duplicated().sum()

0

In [13]:
df = df.drop_duplicates().reset_index(drop=True)

In [14]:
df.duplicated().sum()

0

#### **Вывод:**
Количество дупликатов в датасете 71. Значение не велико и не должно повлиять на анализ.

### Лемматизация

In [15]:
from pymystem3 import Mystem
m = Mystem()

In [16]:
# ознакомимся с целями кредита
df['purpose'].value_counts().sort_index()

автомобили                                421
автомобиль                                454
высшее образование                        413
дополнительное образование                414
жилье                                     587
заняться высшим образованием              440
заняться образованием                     357
на покупку автомобиля                     442
на покупку подержанного автомобиля        437
на покупку своего автомобиля              452
на проведение свадьбы                     685
недвижимость                              572
образование                               405
операции с жильем                         579
операции с коммерческой недвижимостью     581
операции с недвижимостью                  615
операции со своей недвижимостью           559
покупка жилой недвижимости                546
покупка жилья                             595
покупка жилья для сдачи                   588
покупка жилья для семьи                   570
покупка коммерческой недвижимости 

In [17]:
#из данных можно распределить данные на категории: автомобиль, образование, недвижимость, свадьба, если ни одно из слов не подходит - создадим категорию иное
#создадим функцию
def lemm(purpose):
    lemmas = m.lemmatize(purpose)
    if 'жилье' in lemmas or 'недвижимость' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    return 'иное'

In [18]:
df['purpose_category'] = df['purpose'].apply(lemm)
print(df.head(10))

   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0          14177         53   среднее             1   
5         0            926         27    высшее             0   
6         0           2879         43    высшее             0   
7         0            152         50   среднее             1   
8         2           6929         35    высшее             0   
9         0           2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

In [19]:
print(df['purpose_category'].value_counts())

недвижимость    9757
автомобиль      3897
образование     3597
свадьба         2099
Name: purpose_category, dtype: int64


**Вывод:**
- Для удобства анализа мы создали отдельный столбец, где сгруппировали цели кредита.
- Недвижимость, автомобиль, образование и свадьба. самое большое количество в недвижимости.

### Категоризация данных
Для ответа на вопросы нам потребуются данные:
1. Наличие детей
2. Семейное положение
3. Доход
4. Цели кредита


In [20]:
#для категоризации данных дохода, сперва необходимо выявить медиану
df['total_income'].median()

145011.0

In [21]:
# за медиану будем использовать сумму 145 000руб.
#для категоризации используем такие параметры:
# меньше 0,75 медианы (до 108 750руб) - низкий доход
#от 0,75 до 1,25 медианы(от 108 750 до 181 250) - средний доход
#от 1.25 до 2 медиан (от 181 250 до 290 000) - доход выше среднего
# больше двух медиан (от 290 000) - высокий доход
#напишем функцию
def category_total_income(row):
    if row < 108750:
        return 'низкий доход'
    elif row < 181250:
        return 'средний доход'
    elif row < 290000:
        return 'доход выше среднего'
    return 'высокий доход'
df['category_total_income'] = df['total_income'].apply(category_total_income)
print(df['category_total_income'].value_counts())

средний доход          7513
низкий доход           5499
доход выше среднего    4683
высокий доход          1655
Name: category_total_income, dtype: int64


In [22]:
# так как вопрос выставлен о самом факте наличия детей то создадим фунцию и поделим людей на тех у кого есть и нет детей
def category_children(row):
    if row == 0:
        return 'нет детей'
    return 'есть дети'
df['category_children'] = df['children'].apply(category_children)
print(df['category_children'].value_counts())

нет детей    12776
есть дети     6574
Name: category_children, dtype: int64


In [23]:
#во время попытки ответить на вопросы я понял что не удобно когда просроченная задолженность 0 и 1
#по этому создадим новую колонку в которой будет инфо о том есть ли просрочка или нет
def category_debt(row):
    if row == 0:
        return 'нет задолженности'
    return 'задолженность есть'
df['category_debt'] = df['debt'].apply(category_debt)
print(df['category_debt'].value_counts())

нет задолженности     17779
задолженность есть     1571
Name: category_debt, dtype: int64


In [24]:
#для удобства создадим отдельную таблицу только с необходимыми данными
# debt 0 - задолженность отсутствует
df_for_analysis = df[['category_children', 'family_status', 'category_total_income', 'purpose_category', 'category_debt']]
print(df_for_analysis.head())

  category_children     family_status category_total_income purpose_category  \
0         есть дети   женат / замужем   доход выше среднего     недвижимость   
1         есть дети   женат / замужем         средний доход       автомобиль   
2         нет детей   женат / замужем         средний доход     недвижимость   
3         есть дети   женат / замужем   доход выше среднего      образование   
4         нет детей  гражданский брак         средний доход          свадьба   

       category_debt  
0  нет задолженности  
1  нет задолженности  
2  нет задолженности  
3  нет задолженности  
4  нет задолженности  


**Вывод:**
- Для анализа данных нам необходимы были:
- Данные о наличии детей (сгруппировали, т.к. вопрос стоит "влияет ли наличие детей" по этому их количество роли не играет)
- Семейное положение (данный столбец уже имеется)
- Доход (сгруппировали на 4 категории, низкий, средний, выше среднего и высокий доход)
- Цель кредита (сгруппировали ранее)
- Наличие задолженности (заменили с 0 и 1 на нет задоленности и есть (сделал для себя больше, нули и единицы не удобны...))

## Шаг 3. Ответьте на вопросы

Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
#display(df_for_analysis.head())
print(df_for_analysis['category_debt'].value_counts().sum())


19350


In [26]:
children_pivot = df_for_analysis.pivot_table(index='category_children', columns='category_debt', values='family_status', aggfunc='count')
children_pivot['ratio'] = (children_pivot['задолженность есть'] / (children_pivot['нет задолженности'] + children_pivot['задолженность есть'])) * 100
print(children_pivot)

category_debt      задолженность есть  нет задолженности     ratio
category_children                                                 
есть дети                         611               5963  9.294189
нет детей                         960              11816  7.514089


**Вывод:**
- Исходя из данных больше просроченной задолженности у людей с детьми


Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
df_for_analysis.head()

,category_children,family_status,category_total_income,purpose_category,category_debt
0,есть дети,женат / замужем,доход выше среднего,недвижимость,нет задолженности
1,есть дети,женат / замужем,средний доход,автомобиль,нет задолженности
2,нет детей,женат / замужем,средний доход,недвижимость,нет задолженности
3,есть дети,женат / замужем,доход выше среднего,образование,нет задолженности
4,нет детей,гражданский брак,средний доход,свадьба,нет задолженности


In [28]:
family_pivot = df_for_analysis.pivot_table(index='family_status', columns='category_debt', values='category_children', aggfunc='count')
family_pivot['ratio'] = (family_pivot['задолженность есть'] / (family_pivot['нет задолженности'] + family_pivot['задолженность есть'])) * 100

print(family_pivot)

category_debt          задолженность есть  нет задолженности      ratio
family_status                                                          
в разводе                              76               1007   7.017544
вдовец / вдова                         56                809   6.473988
гражданский брак                      339               3395   9.078736
женат / замужем                       846              10297   7.592210
не женат / не замужем                 254               2271  10.059406


**Вывод:**
- Больше всего задолженностей по кредитам у клиентов которые не в браке либо находятся в гражданском браке.
- Далее следуют те кто в официальном браке.
- Меньше всего задолженностей у клиентов в разводе и вдовцов

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [29]:
category_total_income_pivot = df_for_analysis.pivot_table(index='category_total_income', columns='category_debt', values='category_children', aggfunc='count')
category_total_income_pivot['ratio'] = (category_total_income_pivot['задолженность есть'] / (category_total_income_pivot['нет задолженности'] + category_total_income_pivot['задолженность есть'])) * 100
print(category_total_income_pivot)

category_debt          задолженность есть  нет задолженности     ratio
category_total_income                                                 
высокий доход                         120               1535  7.250755
доход выше среднего                   353               4330  7.537903
низкий доход                          438               5061  7.965085
средний доход                         660               6853  8.784773


**Вывод:**
- Больше всего должников в сегменте среднего дохода. 
- Меньше всего в сегменте высокого дохода(не удивительно, таким клиентам нельзя терять лицо) 

Как разные цели кредита влияют на его возврат в срок?

In [30]:
purpose_category_pivot = df_for_analysis.pivot_table(index='purpose_category', columns='category_debt', values='category_children', aggfunc='count')
purpose_category_pivot['ratio'] = (purpose_category_pivot['задолженность есть'] / (purpose_category_pivot['нет задолженности'] + purpose_category_pivot['задолженность есть'])) * 100
print(purpose_category_pivot)

category_debt     задолженность есть  нет задолженности     ratio
purpose_category                                                 
автомобиль                       367               3530  9.417501
недвижимость                     715               9042  7.328072
образование                      331               3266  9.202113
свадьба                          158               1941  7.527394


**Вывод:**
- Здесь хочу отметить что количество приобретающих недвижимость в 3 раза больше любой другой категории, но при этом самый минимальный процент просрочек. Это связано с тем что недвижимость находится в залоге и в случае чего банк в праве изъять данную недвижимость (следовательно у банка больше страховка)
- При этом на цель кредита "авто" самое большое число людей с просрочками, хотя авто так же находится в залоге у банка. 
- Цель "образование" так же находится в высокорискованном сегменте

## Общий вывод

Исходя из данных можно сделать несколько выводов.
1. Наличие детей негативно влияет на возврат кредита. Вероятнее всего это из-за дополнительных семейных затрат
2. Подавляющее число кредитов берут люди в браке и у них один из лучших показателей. Хуже всех возвращают кредиты не в браке или в гражданском браке. Это может быть из-за осознания ответсчтвенности не только за себя. Так же и обязательства по кредиту распределяются на обоих супругов
3. Чем больше доход тем меньше просроченных задолженностей.
4. Меньше всего людей с просроченными задолженностями у людей, целль кредита недвижимое имущество и свадьбы. 
Недвижимость, приобретаемая в кредит находится в залоге у банка до момента полного погашения, по этому для людей с большими просроченными задолженностями есть риск утраты этой недвижимости.
Свадьбы. Кредит на данную цель зачастую гасится благодаря семьям. (не могу ничего сказаьть про эту цель кредита...)
Кредит на образование. Есть риск невозврата данного кредита по причине того что после учебы не всегда есть возможность найти высокооплачиваемую работу.
Кредит на автомобиль. Хотя он и находится в залоге банка но в теории у людей есть возможность реализовать (продать) машину до момента погашения кредита и в случае чего уже у другого человека банк отберет а/м. Самый рискованный кредит для банка. Многие уже отказались от них в пользу простых потребительских кредитов на любые цели.




Проанализировав данные мы можем составить портрет "Идеального клиента":
Без детей
Семейное положение - в разводе или вдовец / вдова или женат / замужем    
Доход - высокий или выше среднего
Цель кредита - приобретение недвижимости


